# Tutorial #1: APIを用いたeLabFTWのノート作成

<b>Note:</b> ご不明な点がございましたら、lee.jemyung@naist.ac.jp にご連絡ください

## **Step 5:** APIを使って保存された実験ノートをダウンロードしよう
#### 記録された実験ノートをPDF、ELN、JSON形式のファイルにダウンロードするサンプルコード

#### Note: _Step 5_ で<span style='color: orange'>新しくお知らせする部分のタイトル</span>はオレンジ色で表示し、その他の部分では _Step 1~4_ と同じ内容となります。

## 前準備
###　「.env」ファイルの作成
「.env」ファイルの作成に関しては[**<ins>こちら</ins>**](https://github.com/naist-eln/eln/blob/main/API/eLabFTW_API_env.ipynb)をご覧ください。<br>
「.env」ファイルがすでに用意されている場合は、そのまま次に進んでください。

### <span style='color: orange'>ダウンロードフォルダの指定</span>
実験ノートをダウンロードするフォルダを指定<br>

In [6]:
# 指定されたフォルダが存在しない場合は、以降のコードを使用して自動的にフォルダを作成

file_path = './test_file/downloads/'

### コード駆動準備
ライブラリ、パッケージ、モジュールの呼び出しと設定

In [1]:
import os
import re
import csv
import datetime
import elabapi_python
from elabapi_python.rest import ApiException
import pandas as pd
from dotenv import load_dotenv
import warnings
from urllib3.exceptions import InsecureRequestWarning

# APIクライアントの設定
load_dotenv()
configuration = elabapi_python.Configuration()
configuration.host = os.environ["HOST_URL"]
configuration.api_key['Authorization'] = os.environ["API_KEY"]
configuration.debug = False
configuration.verify_ssl = False

# SSL警告を無効化
warnings.simplefilter('ignore', InsecureRequestWarning)

# 各種のインスタンスを作成
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name = 'Authorization', header_value = configuration.api_key['Authorization'] )
experimentsApi = elabapi_python.ExperimentsApi(api_client)
templatesApi = elabapi_python.ExperimentsTemplatesApi(api_client)
uploadsApi = elabapi_python.UploadsApi(api_client)
itemsApi = elabapi_python.ItemsApi(api_client)

## 実験ノートデータ取得

### 実験ノートリストの確認
作成した全実験ノートを出力する。

In [10]:
# 全実験ノートの確認
try:
    # デフォルト設定で実験ノートを取得
    print("デフォルト設定で実験ノートを取得:")
    experimentsList = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experimentsList)}")
    
    # 必要なフィールドを抽出してリストに格納
    exp_data = []
    for experiment in experimentsList:
        exp_dict = experiment.to_dict()
        exp_data.append({
            '_date': exp_dict.get('_date'),
            'category_title': exp_dict.get('category_title'),
            'category': exp_dict.get('category'),
            'fullname': exp_dict.get('fullname'),
            'id': exp_dict.get('id'),
            'tags': exp_dict.get('tags'),
            'tags_id': exp_dict.get('tags_id'),
            'up_item_id': exp_dict.get('up_item_id')
        })

    # データフレームに変換
    experiments_df = pd.DataFrame(exp_data)
    experiments_df['_date'] = pd.to_datetime(experiments_df['_date'])   # _date列をdatetime型に変換
    experiments_df = experiments_df.sort_values(by='_date', ascending=False)    # 作成日が新しい順にソート
    # print(experiments_df)
    
except ApiException as e:
    print("ExperimentsApiを呼び出す際の例外: %s\n" % e)

デフォルト設定で実験ノートを取得:
Number of experiments: 22


### 最新の実験ノートを取得
実験ノートリストから最新の実験ノートを取得する。</br>
取得した実験ノートのIDが表示されます。

In [2]:
experiment_id = ''
experiment_title = ''

try:
    experiments_list = experimentsApi.read_experiments()
    
    # 最も新しい実験ノートを取得
    if experiments_list:
        experiments_list = sorted(experiments_list, key=lambda x: x._date, reverse=True)  # 作成日でソート
        experiment_id = experiments_list[0].id  # 最新の実験ノートのIDを取得
        experiment_title = experiments_list[0].title
        print(f"Latest experiment ID: {experiment_id}")        
    else:
        print("No experiments found.")
        exit()

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->read_experiments: {e}\n")
    exit()

Latest experiment ID: 2071


## <span style='color: orange'>実験ノートダウンロード</span>

### <span style='color: orange'>ダウンロードの準備</span>
保存する実験ノートのファイル名の形式を決め、保存するフォルダーを準備します


In [ ]:
# 実験ノートファイルは、「(実験ノートID)_(実験ノートタイトル)_(現在時刻)」の形式で保存されます。

cur_time = datetime.datetime.today()    # システムから現在時刻を取得
output_file = f'{file_path}{experiment_id}_{experiment_title}_{cur_time.strftime("%Y-%m-%d_%H-%M-%S")}_export'

os.makedirs(file_path, exist_ok = True)     # ダウンロード フォルダーの作成

### <span style='color: orange'>ダウンロード実行</span>
PDF、ELN、JSON形式でそれぞれの実験ノートをダウンロードします。

In [7]:
# PDF export
pdf_file_name = f'{output_file}.pdf'
with open(pdf_file_name, 'wb') as pdf_file:
    pdf_file.write(experimentsApi.get_experiment(experiment_id, format='pdf', _preload_content=False).data)
    print(f'A PDF file has been extracted: {pdf_file_name}')
    
# ELN export
eln_file_name = f'{output_file}.eln'
with open(eln_file_name, 'wb') as eln_file:
    eln_file.write(experimentsApi.get_experiment(experiment_id, format='eln', _preload_content=False).data)
    print(f'An ELN file has been extracted: {eln_file_name}')
    
# JSON export
json_file_name = f'{output_file}.json'
with open(json_file_name, 'wb') as json_file:
    json_file.write(experimentsApi.get_experiment(experiment_id, format='json', _preload_content=False).data)
    print(f'A JSON file has been extracted: {json_file_name}')
    
## CSV export
## 注：CSV形式での保存はeLabFTWのAPI機能が不完全であり、必要に応じて別途のコード作成が必要
# csv_file_name = f'{output_file}.csv'
# with open(csv_file_name, 'wb') as csv_file:
#     csv_file.write(experimentsApi.get_experiment(experiment_id, format='csv', _preload_content=False).data)
#     print(f'A CSV file has been extracted: {csv_file_name}')

A PDF file has been extracted: ./test_file/downloaded/2071_Test_No.2_1002_2024-12-10_14-26-19_export.pdf
A CSV file has been extracted: ./test_file/downloaded/2071_Test_No.2_1002_2024-12-10_14-26-19_export.csv
An ELN file has been extracted: ./test_file/downloaded/2071_Test_No.2_1002_2024-12-10_14-26-19_export.eln
A JSON file has been extracted: ./test_file/downloaded/2071_Test_No.2_1002_2024-12-10_14-26-19_export.json


ダウンロードが完了したら、保存されているファイルを確認してみましょう。<br>
次のステップでは、特定のファイルまたはフォルダーの変化を検出して、実験ノートの内容を更新するサンプルコードを見てみましょう。